Train model

In [1]:
# Provides similar functionality to ImageDataGenerators for videos
!pip install keras-video-generators

  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.14-cp37-none-any.whl size=12884 sha256=8c0b61d117c887d0c7c61ed8360954bc025ea995903cb182f37c14b9be438943
  Stored in directory: /root/.cache/pip/wheels/20/b7/76/8674d46fc4777c09e5aa7b065d4e356d90f12ec409a6144bbb
Successfully built keras-video-generators


In [2]:
# Please email me at nini16@tamu.edu if you do not ave access to the google drive.
# Permissions should have been granted but if not please email me!

from google.colab import drive
drive.mount('/content/drive/')
# drive.flush_and_unmount()

Mounted at /content/drive/


In [3]:
# training data.
# Please ensure the file is present before running.
!unzip -q "/content/drive/MyDrive/CSCE636/v6/dataset_v6.zip"

In [2]:
import keras
from keras.regularizers import l2
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Dense, Dropout, Bidirectional
from keras.preprocessing.image import ImageDataGenerator
from keras_video import VideoFrameGenerator

from keras.layers import TimeDistributed, GRU, Dense, Dropout, LSTM

from keras.models import load_model, Model

from keras.applications import ResNet50V2, DenseNet121

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os
import numpy as np

import math

In [7]:
# All frames should be resized
# Please select a batch-size that divides the number of samples!
# THE DATASET WILL PROBABLY BE INCREASED FOR THE NEXT SUBMISSION SO
# BE SURE TO ADJUST THE BATCH SIZE!!

img_shape = (224, 224)
BS = 16

In [8]:
# Apply image augmentation to each frame
# Please confirm that this directory is present before running

vid_gen = VideoFrameGenerator(
    glob_pattern=r"/content/dataset_v6/{classname}/*",
    nb_frames=20, 
    shuffle=True,
    split_val=0.105,
    batch_size=BS,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

class Reading, validation count: 82, train count: 704
class z_miscellaneous, validation count: 78, train count: 672
Total data: 2 classes for 1376 files for train


In [9]:
validation_gen = vid_gen.get_validation_generator()

Total data: 2 classes for 160 files for validation


In [10]:
# model structure for Feature Extractor
def build_convnet_3(shape=(224, 224, 3)):
    resnet = DenseNet121(include_top=False, weights='imagenet', input_shape=shape)

    train = False
    for layer in resnet.layers:
        layer.trainable = train
        if layer.name == "conv5_block3_2_relu":
            train = True
    
    globMaxpool = GlobalMaxPool2D()(resnet.output)
    model = Model(inputs=resnet.input, outputs=globMaxpool)
    return model

In [11]:
def action_model(shape=(20,) + img_shape + (3,)):
    # Create our feature extractor convnet with img_shape input shape
    convnet = build_convnet_3()
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with img_shape shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # add GRU
    model.add(LSTM(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(l2=0.01)))
    model.add(Dropout(.5))
    model.add(Dense(2, activation='softmax'))

    model.summary()
    return model

In [12]:
# instantiate and compile model
model = action_model()
optimizer = keras.optimizers.Adam(0.0005) #0.0005
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

29089792/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 1024)          7037504   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                278784    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 7,384,898
Trainable params: 347,394
Non-trainable params: 7,037,504
_________________________________________________________________


In [ ]:
# Adjust epochs and other parameters as needed
# Callbacks have been commented out to avoid overwriting existin data.
# Whoever is running this can uncomment them as needed

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.0001),
    keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=5,
        ),
    keras.callbacks.ModelCheckpoint(
        r'C:\Users\cotua\Desktop\python scripts\Weights\weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        monitor='val_acc',
        save_best_only=True,
        verbose=1),
]

history = model.fit_generator(
    vid_gen,
    steps_per_epoch=math.ceil(vid_gen.files_count/BS),
    validation_data=validation_gen,
    verbose=1,
    epochs=30, # last used 80,
    shuffle=True,
    callbacks=callbacks
)

c:\users\cotua\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
86/86 [==============================] - 321s 4s/step - loss: 1.5203 - acc: 0.6640 - val_loss: 0.7254 - val_acc: 0.9062

Epoch 00001: val_acc improved from -inf to 0.90625, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.01-0.73.hdf5
Epoch 2/30
86/86 [==============================] - 82s 956ms/step - loss: 0.6511 - acc: 0.8896 - val_loss: 0.4675 - val_acc: 0.9125

Epoch 00002: val_acc improved from 0.90625 to 0.91250, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.02-0.47.hdf5
Epoch 3/30
86/86 [==============================] - 82s 954ms/step - loss: 0.4352 - acc: 0.9194 - val_loss: 0.3599 - val_acc: 0.9250

Epoch 00003: val_acc improved from 0.91250 to 0.92500, saving model to C:\Users\cotua\Desktop\python scripts\Weights\weights.03-0.36.hdf5
Epoch 4/30
86/86 [==============================] - 82s 958ms/step - loss: 0.3185 - acc: 0.9412 - val_loss: 0.3275 - val_acc: 0.8938

Epoch 00004: val_acc did not improve from 0.92500
Epoch

In [ ]:
# uncomment only if you need to
model.save(r'C:\Users\cotua\Desktop\python scripts\new_model_V6_lr_0.0005.h5')

In [ ]:
# uncomment only if you need to
np.save(r'C:\Users\cotua\Desktop\python scripts\train_history_new_model_V6_lr_0.0005.npy',history.history)

**Testing model**

In [3]:
model=load_model(r'/content/drive/MyDrive/CSCE636/v6/new_model_V6_lr_0.0005.h5')

In [4]:
!unzip -q "/content/drive/MyDrive/CSCE636/v5/youtube-neg-test.zip"

In [5]:
!unzip -q "/content/drive/MyDrive/CSCE636/v6/youtube_positive.zip"

Positive Youtube Videos

In [8]:
test_gen = VideoFrameGenerator(
    glob_pattern=r"/content/youtube_positive/{classname}/*",
    nb_frames=20, 
    shuffle=False,
    batch_size=15,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

Total data: 2 classes for 1665 files for train


In [9]:
len(test_gen)

111

In [14]:
# Run for only 5 steps since google colab cant handle entire test set
model.evaluate_generator(test_gen, steps=len(test_gen))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.4872882068157196, 0.8999999761581421]

Negative Youtube Videos

In [15]:
test_gen = VideoFrameGenerator(
    glob_pattern=r"/content/youtube-neg-test/{classname}/*",
    nb_frames=20, 
    shuffle=True,
    batch_size=10,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      samplewise_center=True,
                                      # rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      # zoom_range=0.1,
                                      # horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

Total data: 2 classes for 310 files for train


In [12]:
len(test_gen)

31

In [17]:
model.evaluate_generator(test_gen, steps=len(test_gen))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.46209976077079773, 0.8600000143051147]